In [8]:
#Advanced Text Preprocessing
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import random
import nltk
import os
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, cohen_kappa_score
from sklearn.model_selection import GroupKFold, train_test_split
from sklearn import tree, metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


random.seed(20)

# Load spaCy model for advanced tokenization and lemmatization
nlp = spacy.load('en_core_web_sm')

# Load dataset
df = pd.read_csv('/Users/emme/Desktop/annotation_final_features.csv')
column_names = df.columns
print(column_names)

# split data for student level cross validation: one student is nested within one fold
group_dict = dict()
groups = np.array([])

for index, row in df.iterrows():
    s_id = row['created_by']
    if s_id not in group_dict:
        group_dict[s_id] = index
    groups = np.append(groups, group_dict[s_id])
    
# Set up the splitter with 5 splits
gkf = GroupKFold(n_splits = 5)

# Function to preprocess text data
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

# Apply preprocessing to the text data
df['preprocessed_text'] = df['annotation_text'].apply(preprocess_text)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(df['preprocessed_text']).toarray()

Index(['thinklet_id', 'problem_name', 'created_by', 'notify_to', 'created_at',
       'annotation_text', 'relating_to_self', 'comment_answer',
       'comment_process', 'count_nChar', 'count_nWords', 'contain_question',
       'contain_plan', 'contain_addTransition', 'contain_advTransition',
       'contain_causalTransition', 'contain_seqTransition', 'count_nNouns',
       'count_nPronouns', 'count_nVerbs', 'count_nAdj', 'count_nAux',
       'count_nCconj', 'count_nPunct', 'count_nSym', 'count_nFPP',
       'contain_same', 'contain_answer', 'contain_correct', 'count_nPositive',
       'count_nNegative'],
      dtype='object')


In [9]:
#Neural Network Architecture
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Neural network with additional layers and dropout
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [15]:
#Training with Early Stopping and Hyperparameter Tuning
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# stemming
porter = PorterStemmer() # stemming recovers root words from plurals etc
stemmed_texts = []

X_text = df['annotation_text']

for answer in X_text:
    answer = ' '.join(porter.stem(word) for word in answer.split(' '))
    stemmed_texts.append(answer)

X, y = np.array(stemmed_texts), df['relating_to_self'] # CHANGE y HERE

#vect = CountVectorizer(ngram_range=(1,1), max_features=1000, stop_words="english") #only unigram 313 features
vect = CountVectorizer(ngram_range=(2,2), max_features=1000, stop_words="english") #only bigram 728 features

X = vect.fit_transform(X).toarray()


# set up storage arrays for each round of validation
roc_auc_scores = np.array([])
accuracy_scores = np.array([])

# split, train, test and store performance metrics
for train_index, test_index in gkf.split(X, y, groups=groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    model = Sequential()
    model.add(Dense(12, input_shape=(738,), activation='relu')) 
    model.add(Dense(8, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics = ['acc']
             )
    
    num_epochs = 30
    batch_size = 10

    model.fit(
        X_train, 
        y_train, 
        epochs=num_epochs, 
        validation_split=0.1,
        shuffle=True, 
        batch_size=batch_size, callbacks=[early_stopping])
    
    predictions = model.predict(X_test)
    
    # compute some metrics and store them for averaging later on
    roc_auc_scores = np.append(roc_auc_scores, roc_auc_score(y_test, predictions))

# print mean scores for the 5-fold CV
print("average roc_auc score: ", np.round(roc_auc_scores.mean(), 3))
print("stdv roc_auc score: ", np.round(roc_auc_scores.std(), 3))
print("max roc_auc score: ", np.round(roc_auc_scores.max(), 3))

Epoch 1/30
17/17 [==============================] - 0s 5ms/step - loss: 0.6785 - acc: 0.6768 - val_loss: 0.6681 - val_acc: 0.6842
Epoch 2/30
17/17 [==============================] - 0s 1ms/step - loss: 0.6404 - acc: 0.7744 - val_loss: 0.6395 - val_acc: 0.7895
Epoch 3/30
17/17 [==============================] - 0s 1ms/step - loss: 0.6015 - acc: 0.7805 - val_loss: 0.6071 - val_acc: 0.7895
Epoch 4/30
17/17 [==============================] - 0s 1ms/step - loss: 0.5591 - acc: 0.7683 - val_loss: 0.5735 - val_acc: 0.7895
Epoch 5/30
17/17 [==============================] - 0s 1ms/step - loss: 0.5109 - acc: 0.7866 - val_loss: 0.5389 - val_acc: 0.7895
Epoch 6/30
17/17 [==============================] - 0s 1ms/step - loss: 0.4583 - acc: 0.8049 - val_loss: 0.5058 - val_acc: 0.7895
Epoch 7/30
17/17 [==============================] - 0s 1ms/step - loss: 0.4064 - acc: 0.8659 - val_loss: 0.4729 - val_acc: 0.8421
Epoch 8/30
17/17 [==============================] - 0s 1ms/step - loss: 0.3547 - acc: 0.89